# GMAP package metadata form

| Field | Field description (and example entries) |
| --- | --- |
| Map name (GMAP_ID) | Unique package name (`GMAP-{{target-body}}-{{content-type}}-{{label}}`) |
| Target body | Name of target body (eg, `Mercury`) |
| Title of map | Map title (eg, `Awesome Geologic Map of the region X`) |
| Bounding box - Min Lat | Minimum latitude in degrees [-90:90) (< Max Lat) |
| Bounding box - Max Lat | Maximum latitude in degrees (-90:90] (> Min Lat) |
| Bounding box - Min Lon | West-most Longitude in degrees [-180:180) (< Max Lon) |
| Bounding box - Max Lon | East-most Longitude in degrees (-180:180] (> Min Lon) |
| Author(s) | Semi-colon separated list of authors |
| Type | Either "draft" or "released" |
| Output scale | Map spatial scale |
| Original Coordinate Reference System | WKT declaring map' CRS |
| Data used | Semi-colon separated list of ancillary, original data used |
| Standards adhered to | Semi-colon list of standards used in the map |
| DOI of companion paper(s) | DOI of linked publication |
| Aims | Reason, goal for this map |
| Short description | Free-text (500 words maximum) describing the map |
| Related products | Other geological maps complementing this one |
| Units Definition (polygon styling) | Units color definition |
| Stratigraphic info | Description of stratigraphic elements in the map |
| Other comments | free-text (notes, errata, warnings) |
| Heritage used | heritage information |
| Link to other data | Links to extenal resources |
| Acknowledgements | Free-text acknowledge |

## Form schema

The idea of the _Form_ is to use a JSON-schema (see https://json-schema.org) to set the form widget. 
The metadata fields (or _attributes_) can have different types and we want the form to represent each with the apropriate widget.
The use of schema to formally define the (metadata) attributes and have an interface that understands it to provide a graphical interface gives a lot of flexibility in the managing of data packages.

The `schema` below represent a GMAP package metadata (above).

In [1]:
# The schema for GMAP metadata (https://wiki.europlanet-gmap.eu/bin/view/Main/Documentation/Map-wide%20metadata/).
#
schema = {
    "type": "object",
    
    "properties": {
        'title': {
            "type": "string",
            "description": "Map title"
        },

        'authors': {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "lastname": {"type": "string"},
                    "firstname": {"type": "string"}
                }
            },
            "minItems": 1
        },

        'bbox_lon_west': {
            "description": "West Longitude",
            "type": "number",
            "minimum": -180,
            "maximum": 180
        }, 

        'bbox_lon_east': {
            "description": "East Longitude",
            "type": "number",
            "minimum": -180,
            "maximum": 180
        },

        'bbox_lat_min': {
            "description": "Min Latitude",
            "type": "number",
            "minimum": -90,
            "maximum": 90
        }, 

        'bbox_lat_max': {
            "description": "Max Latitude",
            "type": "number",
            "minimum": -90,
            "maximum": 90
        }, 

        'publication_date': {
            "description": "Publication date",
            "type": "string",
            "format": "date"
        },

        'doi': {
            "type": "string",
        },

        'crs': {
            "type": "string",
        },

         'shortname': {
            "type": "string"
        },

        'map_type': {
            "description": "Map type",
            "type": "array",
            "items": {
                "type": "string",
                "enum": [
                    "Integrated",
                    "Morphologic",
                    "Compositional",
                    "Digital model",
                    "Stratigraphic",
                    "Geo-structural",
                ]
            },
            "minItems": 1,
            "uniqueItems": True
        },

        'target': {
            "description": "Target body",
            "type": "string",
            "enum": ['Mars', 'Mercury', 'Moon', 'Venus']
        },

       'gmap_id': {
            "type": "string",
            "readOnly": True
        }
    },
    
    "required": [
        "gmap_id",
        "target",
        "map_type",
        "shortname",
        "title",
    ]
}

In [2]:
from api.gui import form

form = form.Form(schema=schema)

In [3]:
# Optionally, we can layout the attribute (widgets) in some groups.
# THE LAYOUT IS A LIST of strings, lists, or dictionaries. The strings are the (schema) attribute names;
# Lists will compose box widgets (`VBox`) with _strings, lists, or dictionaries_;
# The dictionaries key is "the group label", and a _string, list, or dictionary_ are the possible values.
# This structure can be as recursive (nested) as necessary.

layout = [
    'title',
    'shortname',
    'map_type',
    'target',
    [
        {"Longitude (west,east) [-180:180]": ['bbox_lon_west', 'bbox_lon_east']},
        {"Latitude (min,max) [-90:90]": ['bbox_lat_min', 'bbox_lat_max']}
    ],
    'publication_date',
    'doi',
    'authors',
    [
        # {"Map description": ['description', 'aims', 'units', 'stratigraphic_info']},
        # {"Spatial attributes": ['crs', 'output_scale']},
        # {"Ancillary data": ['ancillary_data', 'related_products', 'heritage', 'extra_data']},
        # {"Notes": ['standards', 'comments', 'acknowledge']}
    ],
    'gmap_id'
]


form.set_layout(layout)

In [4]:
from IPython.display import display, JSON


def on_change_value(change):
    """
    Set (or reset) 'gmap_id' when 'target','map_type','shortname' change values
    """
    map_types = {
        "Integrated": "I",
        "Morphologic": "M",
        "Compositional": "C",
        "Digital model": "D",
        "Stratigraphic": "S",
        "Geo-structural": "G",
    }

    _target = form['target'].value.strip()
    _type = ''.join([ map_types[t].upper() for t in form['map_type'].value ])
    _sname = form['shortname'].value.strip().replace(' ','-')
    _id = ["GMAP", _type, _target, _sname]
    
    if not all(_id):
        form['gmap_id'].value = ""
    else:
        form['gmap_id'].value = '_'.join(_id)

        
# Link the widgets to trigger the function above ('on_change_value')
#
form['target'].observe(on_change_value, names='value')
form['shortname'].observe(on_change_value, names='value')
form['map_type'].observe(on_change_value, names='value')

# form.set_link(source=['target','map_type','shortname'], destiny='gmap_id', trigger=on_change_value)

In [5]:
from ipywidgets import widgets

output_panel = widgets.Output(layout={'border': '1px solid black'})

@output_panel.capture(clear_output=True)
def validate_btn(*args,**kargs):
    import json
    # print(json.dumps(form.to_json(), indent=2))
    display(JSON(form.to_json()))
        
btn_to_json = widgets.Button(icon="save", description="")
btn_to_json.on_click(validate_btn)

output_json = widgets.VBox([btn_to_json, output_panel])

In [6]:
from ipywidgets import widgets

app = widgets.AppLayout(
    header = None,
    left_sidebar = form.widget,
    center = None,
    right_sidebar = output_json,
    footer = None
)

display(app)

AppLayout(children=(VBox(children=(Text(value='', description='<strong style="color:red">Title</strong>'), Tex…

In [7]:
form.read_json({
    'shortname': 'Some name',

    'map_type': [
                "Integrated",
                "Morphologic",
                "Compositional",
                "Digital model",
                "Stratigraphic",
                "Geo-structural",
            ],

    'target': 'Mars',
    
    'authors': {'firstname':'zezinho'},

    'title': 'A map'
})

In [9]:
form.set('doi','bla')

form.set('target','Venus')

form.set('shortname','Some region')

form.set('map_type',['Morphologic','Stratigraphic'])

form.set('authors',{'firstname':'bla','lastname':'ble'})

In [ ]:
from pprint import pprint
pprint(form)

In [ ]:
form.to_json()

In [ ]:
# with output_panel:
#         print(form.to_json())
        
